## 測驗題1

### Web Scraping

In [1]:
# web scraping
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# time wait
import time

# dataframe
import pandas as pd

/Users/demi/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
# function for option selection
def select_option(element, option):
    select = Select(element)
    select.select_by_visible_text(option)

In [3]:
area = '台北'
depart = '臺北轉運站'
dest = '朝馬轉運站'

In [4]:
# build a chrome webdriver
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.implicitly_wait(3)
browser.get('http://www.kingbus.com.tw/ticketRoute.php')

## select options

# first select area
area_element = browser.find_element(By.ID, 'area')
select_option(area_element, area)

# wait for the output's ready
browser.implicitly_wait(3)

# then select departure and destination
depart_element = browser.find_element(By.ID, 'origin')
select_option(depart_element, depart)

dest_element = browser.find_element(By.ID, 'destination')
select_option(dest_element, dest)

# click submit button and wait for the results
submit = browser.find_element(By.NAME, 'submit')
submit.click()

# get the results
result = browser.find_element(By.CLASS_NAME, 'routeData')
#print(result.text)


[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [/Users/demi/.wdm/drivers/chromedriver/mac64/95.0.4638.54/chromedriver] found in cache
/Users/demi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


### Output the results to CSV

In [5]:
# build csv
csv_rows = []
for s in result.text.split('\n'):
    s_ = s.split('\u3000')
    cln_first = s_[0].split(' ')[1]
    cln_second = ''.join(s_[1:]).replace(' ', '')
    csv_rows.append([cln_first, cln_second])
    
df_route = pd.DataFrame(csv_rows, columns=['route', 'description'])
df_route.to_csv('route.csv', index = False)

In [6]:
# prepare data for sql
sql_data = list(tuple(i) for i in csv_rows)
#print(len(sql_data))

### Write Data to SQL Server and Count '埔里' in routes

In [7]:
import mysql.connector

In [8]:
# Connect MySQL server

maxdb = mysql.connector.connect(
  host = "127.0.0.1",
  user = "root",
  password = "a123456**",
  )
cursor=maxdb.cursor()

In [9]:
# Create DB
cursor.execute("CREATE DATABASE bus_route")

In [10]:
# connect to MySQL DB
maxdb = mysql.connector.connect(
  host = "127.0.0.1",
  user = "root",
  password = "a123456**",
  database = "bus_route",
  )
cursor=maxdb.cursor()

# Create table
cursor.execute("CREATE TABLE bus_routes (route VARCHAR(99), description VARCHAR(99), id INTEGER AUTO_INCREMENT PRIMARY Key)")

# Insert results
sqlStuff = "INSERT INTO bus_routes (route, description) VALUES (%s,%s)"

cursor.executemany(sqlStuff, sql_data)
maxdb.commit()

In [11]:
 # Read
cursor.execute("SELECT * FROM bus_routes Where route Like '%埔里%'")
select_result = cursor.fetchall()
#for row in select_result:
#    print(row)
print(f'路線中含埔里次數： {len(select_result)}次')

路線中含埔里次數： 2次


In [ ]:
# Delete Table
#  delete_table = "DROP TABLE IF EXISTS bus_routes"
#  cursor.execute(delete_table)

In [ ]:
# Delete DB
#  delete_table = "DROP DATABASE IF EXISTS bus_route"
#  cursor.execute(delete_table)